In [1]:
from pathlib import Path
import tensorflow as tf
import numpy as np

# Import the necessary Keras modules containing model and layers
# Used this method to avoid import issueds from pylint
ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator
MobileNetV3Large = tf.keras.applications.MobileNetV3Large
Dense = tf.keras.layers.Dense
Input = tf.keras.layers.Input
Model = tf.keras.models.Model
Adam = tf.keras.optimizers.Adam
image = tf.keras.preprocessing.image

training_path = Path("../../data/two_class_dataset/train/")


# Load the pre-trained MobileNetV3Large model, excluding its top (final) layer.
base_model = MobileNetV3Large(
    weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3))
)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Add new layers on top of MobileNet
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(1, activation="sigmoid")(x)  # Assuming binary classification

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

# Create an instance of ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)

# Assuming your images are in a directory named 'data/train'
train_generator = train_datagen.flow_from_directory(
    training_path,  # This is the target directory
    target_size=(224, 224),  # All images will be resized to 224x224
    batch_size=32,
    class_mode="binary",
)  # Since we use binary_crossentropy loss, we need binary labels
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
)

# Save the model
model.save("my_model")
# To use the model for inference
imape = tf.keras.preprocessing.image


def predict(image_path: Path, model: tf.keras.Model):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Remember to rescale the image as we did for the training data

    # Predict the class
    prediction = model.predict(img_array)
    return prediction[0][0]

2023-11-08 20:25:50.053893: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-08 20:25:50.055828: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-08 20:25:50.086203: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-08 20:25:50.086232: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-08 20:25:50.086250: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

Found 402 images belonging to 2 classes.
Epoch 1/10
12/12 [==============================] - 8s 435ms/step - loss: 0.5843 - accuracy: 0.7514
Epoch 2/10
12/12 [==============================] - 5s 385ms/step - loss: 0.5504 - accuracy: 0.7568
Epoch 3/10
12/12 [==============================] - 5s 384ms/step - loss: 0.5462 - accuracy: 0.7459
Epoch 4/10
12/12 [==============================] - 4s 375ms/step - loss: 0.5252 - accuracy: 0.7486
Epoch 5/10
12/12 [==============================] - 5s 421ms/step - loss: 0.5168 - accuracy: 0.7459
Epoch 6/10
12/12 [==============================] - 5s 410ms/step - loss: 0.4933 - accuracy: 0.7568
Epoch 7/10
12/12 [==============================] - 4s 360ms/step - loss: 0.4786 - accuracy: 0.7595
Epoch 8/10
12/12 [==============================] - 4s 357ms/step - loss: 0.4819 - accuracy: 0.7432
Epoch 9/10
12/12 [==============================] - 5s 398ms/step - loss: 0.4543 - accuracy: 0.7514
Epoch 10/10
12/12 [==============================] - 5s 362

INFO:tensorflow:Assets written to: my_model/assets


In [2]:
validation_path = Path("../../data/two_class_dataset/validate/")

passing = validation_path / "class_1"
failing = validation_path / "class_0"

print(passing)


over_80_passing = [
    image_path
    for image_path in passing.glob("*.png")
    if predict(image_path, model) > 0.8
]

over_80_failing = [
    image_path
    for image_path in failing.glob("*.jpg")
    if predict(image_path, model) < 0.8
]

../../data/two_class_dataset/validate/class_1
1/1 [==============================] - 0s 30ms/step


In [3]:
# for image_path in passing.glob("*.png"):
#     print(image_path, predict(image_path, model))
# for image_path in failing.glob("*.jpg"):
#     print(image_path, predict(image_path, model))
# for image_path in (training_path / "class_1").glob("*.png"):
#     print(image_path, predict(image_path, model))

In [4]:
print(len(over_80_passing))
# print(len(over_80_failing))

162


In [5]:
len(over_80_passing) / len(list(passing.glob("*.png")))

0.5382059800664452